In [1]:
import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import shutil as sh
import math
import time

In [7]:
class vdw_vol():
    def __init__(self):
        self.path='C:\\Users\\koyama\\OSC\\interaction\\anthracene\\vdw\\'
        self.interval_theta=1.0#deg
        self.interval_A=10.0
#         self.A1_list=[33.0]#[round(A1) for A1 in np.linspace(0,50,6)]
#         self.A2_list=[10.,11.,12.,13.,14.,15.]
        #self.A2_list=[round(A2) for A2 in np.linspace(0,90,91)]
        #self.A3_list=[round(A3) for A3 in np.linspace(-90,90,19)]
        #self.theta_list=np.linspace(0,90,90/self.interval_theta+1)
        #self.opt_list=[[3.8,2.9,20],[4.1,2.3,30],[4.3,1.8,40],[4.4,1.5,45],[4.5,1.2,50],[4.6,0.9,55],[4.8,0.5,60],[5.0,-0.2,70],[5.1,-1.0,80],[5.0,0.0,90]]
        
        #step2A+3にむけてのvdw用
        self.df_para_all=pd.read_csv(self.path+'parameter_all.csv')

    def Rod(self,n,theta_in):
        nx,ny,nz=n
        theta_t=np.radians(theta_in)
        Rod=np.array([[np.cos(theta_t)+(nx**2)*(1-np.cos(theta_t)),nx*ny*(1-np.cos(theta_t))-nz*np.sin(theta_t),nx*nz*(1-np.cos(theta_t))+ny*np.sin(theta_t)],
                    [nx*ny*(1-np.cos(theta_t))+nz*np.sin(theta_t),np.cos(theta_t)+(ny**2)*(1-np.cos(theta_t)),ny*nz*(1-np.cos(theta_t))-nx*np.sin(theta_t)],
                    [nx*nz*(1-np.cos(theta_t))-ny*np.sin(theta_t),ny*nz*(1-np.cos(theta_t))+nx*np.sin(theta_t),np.cos(theta_t)+(nz**2)*(1-np.cos(theta_t))]])
        return Rod

# ####平面角がstep1のheriになるA3を出力
#     def A3_to_heri(self,A1,A2,heri):
#         N=181
#         A1=np.radians(A1);A2=np.radians(A2)
#         ax1=np.array([np.sin(A1)*np.cos(A2),np.sin(A1)*np.sin(A2),np.cos(A1)])
#         ax2=np.array([np.sin(A1)*np.cos(A2),-np.sin(A1)*np.sin(A2),np.cos(A1)])
#         heri_list=np.zeros(N,dtype='float64');error_list=np.zeros(N,dtype='float64')
#         A3_list=np.array([round(A3) for A3 in np.linspace(-90,90,N)])
#         for i,A3 in enumerate(A3_list):
#             ex1=np.matmul(self.Rod(ax1,A3),np.array([-np.cos(A1)*np.cos(A2),-np.cos(A1)*np.sin(A2),np.sin(A1)]))
#             ex2=np.matmul(self.Rod(ax2,-A3),np.array([-np.cos(A1)*np.cos(A2),+np.cos(A1)*np.sin(A2),np.sin(A1)]))
#             heri_list[i]=np.degrees(np.arccos(np.dot(ex1,ex2)))
#             error_list[i]=abs(heri_list[i]-heri)
#         idx=np.argsort(error_list);heri_sort=heri_list[idx];A3_sort=A3_list[idx]
#         heri_1,heri_2=A3_sort[0],A3_sort[1]
#         return heri_1,heri_2#-11degか37degか
    
    
#分子1分子2を作成(回転させただけ・平行移動なし)
    def convertor(self,atom_list,A1,A2,A3):
        self.n=np.array([np.sin(np.radians(A1))*np.cos(np.radians(A2)),
                         np.sin(np.radians(A1))*np.sin(np.radians(A2)),
                         np.cos(np.radians(A1))])######回転軸
        atom_list_rt=[]
        for x,y,z,R in atom_list:
            x1,y1,z1=np.matmul(self.Rod(np.array([0,1,0]),A1),np.array([x,y,z]).T)
            x2,y2,z2=np.matmul(self.Rod(np.array([0,0,1]),A2),np.array([x1,y1,z1]).T)
            x3,y3,z3=np.matmul(self.Rod(self.n,A3),np.array([x2,y2,z2]).T)
            atom_list_rt.append([x3,y3,z3,R])
        return np.array(atom_list_rt)

####R返す関数
    def vdw_R(self,A1,A2,A3,theta,mode):
        df_anth=pd.read_csv(self.path+'monomer.csv')###x,y,z,rad
        anth=df_anth[['X','Y','Z','R']].values
        anth_1=self.convertor(anth,A1,A2,A3)
        if mode=='t':
            anth_2=self.convertor(anth,A1,-A2,-A3)#回転させただけ
        elif mode=='a' or mode=='b':
            anth_2=self.convertor(anth,A1,A2,A3)
        R_clps=0
        eR=np.array([np.cos(np.radians(theta)),np.sin(np.radians(theta)),0.0])
        for x1,y1,z1,rad1 in anth_1:
            for x2,y2,z2,rad2 in anth_2:
                R_1b=np.dot(eR,np.array([x1,y1,z1]))
                R_2b=np.dot(eR,np.array([x2,y2,z2]))
                R_12=np.array([x2-x1,y2-y1,z2-z1])
                R_12b=np.dot(eR,R_12)
                R_12a=np.linalg.norm(R_12-R_12b*eR)
                if (rad1+rad2)**2-R_12a**2<0:
                    continue
                else:
                    R_clps=max(R_clps,R_1b-R_2b+np.sqrt((rad1+rad2)**2-R_12a**2))
        return R_clps
    
    def make_csv(self,name_csv,squeeze_theta=True):
        start=time.time()
        df_vdw=pd.DataFrame(columns=['A1','A2','A3','theta','R','S','a','b'])
        for A1 in self.A1_list:
            for A2 in self.A2_list:
                #if A1==0 and A2 >-90 :
                #    continue
                for A3 in self.A3_list:
                    print('A1={} A2={} A3={}'.format(A1,A2,A3))
                    a_clps=self.vdw_R(A1,A2,A3,0.0,'a')
                    b_clps=self.vdw_R(A1,A2,A3,90.0,'b')
                    for theta in self.theta_list:
                        R_clps=self.vdw_R(A1,A2,A3,theta,'t')
                        a=2*R_clps*np.cos(np.radians(theta))
                        b=2*R_clps*np.sin(np.radians(theta))
                        if (a_clps > a) or (b_clps > b):
                            continue
                        else:
                            S=a*b
                            data=pd.Series([A1,A2,A3,theta,R_clps,S,a,b],index=df_vdw.columns)
                            df_vdw=df_vdw.append(data,ignore_index=True)
                            if squeeze_theta:
                                continue
                                
        self.df_vdw=df_vdw
        df_vdw.to_csv(self.path+name_csv)
        end=time.time()
        print('time:{}s'.format(end-start))
    
#     def make_csv_A3heri(self,A1,heri,name_csv):#平面角をヘリンボーン角に保つようA3を決定
#         df_vdw=pd.DataFrame(columns=['A1','A2','A3','theta','R','S','a','b'])
#         for A2 in self.A2_list:
#             heri_1,heri_2=self.A3_to_heri(A1,A2,heri)
#             for A3 in [heri_1,heri_2]:
#                 print('A1={} A2={} A3={}'.format(A1,A2,A3))
#                 a_clps=self.vdw_R(A1,A2,A3,0.0,'a')
#                 b_clps=self.vdw_R(A1,A2,A3,90.0,'b')
#                 for theta in self.theta_list:
#                     R_clps=self.vdw_R(A1,A2,A3,theta,'t')
#                     a=2*R_clps*np.cos(np.radians(theta))
#                     b=2*R_clps*np.sin(np.radians(theta))
#                     if (a_clps > a) or (b_clps > b):
#                         continue
#                     else:
#                         S=a*b
#                         data=pd.Series([A1,A2,A3,theta,R_clps,S,a,b],index=df_vdw.columns)
#                         df_vdw=df_vdw.append(data,ignore_index=True)
#         df_vdw.to_csv(self.path+name_csv)
        
    def make_csv_interlayer(self,A1,A2,A3,a_,b_):#,name_csv
        a=np.array([a_,0,0]);b=np.array([0,b_,0]);t1=(a+b)/2;t2=(a-b)/2
        df_anth=pd.read_csv(self.path+'monomer.csv')###x,y,z,rad
        anth=df_anth[['X','Y','Z','R']].values
#         anth_zp=np.array([[x,y,z,r] for x,y,z,r in anth if z>0])
#         anth_zm=np.array([[x,y,z,r] for x,y,z,r in anth if z<0])
        anth_i0=self.convertor(anth,A1,A2,A3)#層間
        anth_p=self.convertor(anth,A1,A2,A3)#層内
        anth_t=self.convertor(anth,A1,-A2,-A3)#層内
        arr_list=[[np.zeros(3),'p'],[b,'p'],[-b,'p'],[a,'p'],[-a,'p'],[t1,'t'],[-t1,'t'],[t2,'t'],[-t2,'t']]
        Ra_list=[np.round(Ra,1) for Ra in np.linspace(-np.round(a_/2,1),np.round(a_/2,1),int(np.round(2*np.round(a_/2,1)/0.1))+1)]
        z_list=[];V_list=[]
        df_vdw=pd.DataFrame(columns=['Ra','R3','V'])
        for Ra in Ra_list:
            z_max=0
            for R,arr in arr_list:
                if arr=='t':
                    anth=anth_t
                elif arr=='p':
                    anth=anth_p
                for x1,y1,z1,R1 in anth:#層内
                    x1,y1,z1=np.array([x1,y1,z1])+R
                    for x2,y2,z2,R2 in anth_i0:#i0
                        x2+=Ra
                        z_sq=(R1+R2)**2-(x1-x2)**2-(y1-y2)**2
                        if z_sq<0:
                            z_clps=0.0
                        else:
                            z_clps=np.sqrt(z_sq)+z1-z2
                        z_max=max(z_max,z_clps)
            z_list.append(z_max)
            V_list.append(a_*b_*z_max)
        df_vdw['Ra']=Ra_list;df_vdw['R3']=z_list;df_vdw['V']=V_list
        return np.array([Ra_list[np.argmin(V_list)],0,z_list[np.argmin(V_list)]])
        #df_vdw.to_csv(self.path+name_csv)

# ###2/1スライド
#     def make_csv_all_cifA(self,name_csv):
#         df_vdw_all=pd.DataFrame(columns=[['A1','A2','A3','a','b','Ra','R3','V']])
#         time=0
#         A1=30.86249;A2=14.66922;A3=-38.3561;a_cif=8.5576;b_cif=6.0100#Ra=4.3,Rb=3.0
#         Ra_list=[np.round(Ra,1) for Ra in np.linspace(3.8,4.7,10)]
#         Rb_list=[np.round(Rb,1) for Rb in np.linspace(2.5,3.4,10)]
#         for Ra in Ra_list:
#             for Rb in Rb_list:
#                 time+=1
#                 c=self.make_csv_interlayer(A1,A2,A3,2*Ra,2*Rb)
#                 data=pd.Series([A1,A2,A3,2*Ra,2*Rb,c[0],c[2],2*Ra*2*Rb*c[2]],index=df_vdw_all.columns)
#                 df_vdw_all=df_vdw_all.append(data,ignore_index=True)
#                 if time%10==0:
#                     print('{}% completed'.format(time))
#         c=self.make_csv_interlayer(A1,A2,A3,a_cif,b_cif)
#         data=pd.Series([A1,A2,A3,a_cif,b_cif,c[0],c[2],a_cif*b_cif*c[2]],index=df_vdw_all.columns)
#         df_vdw_all=df_vdw_all.append(data,ignore_index=True)
#         df_vdw_all.to_csv(self.path+name_csv)
        
    def make_csv_all(self,name_csv):
        df_vdw_all=pd.DataFrame(columns=[['A1','A2','A3','a','b','Ra','R3','V']])
        time=0
        N=len(self.df_para_all[['A1','A2','A3','Ra','Rb']].values)
        for A1,A2,A3,Ra_,Rb_ in self.df_para_all[['A1','A2','A3','Ra','Rb']].values:
            a_=2*Ra_;b_=2*Rb_
            time+=1
            c=self.make_csv_interlayer(A1,A2,A3,a_,b_)
            data=pd.Series([A1,A2,A3,a_,b_,c[0],c[2],a_*b_*c[2]],index=df_vdw_all.columns)
            df_vdw_all=df_vdw_all.append(data,ignore_index=True)
            print('{}% completed'.format(time*100/N))
        df_vdw_all.to_csv(self.path+name_csv,index=False)    
        
#     def cmap(self,name_csv):##self.listによってカスタマイズする
#         df_vdw=pd.read_csv(self.path+name_csv)
#         plt.rcParams['font.size']=15
#         plt.rcParams['figure.figsize']=8,60
#         ind_fig=1
#         for ind,A2 in enumerate(self.A2_list):
#             para_=[para for para in self.opt_list if para[2]/2==A2]
#             if len(para_)==0:
#                 continue
#             ind_fig+=1
#             plt.subplot(10,1,ind_fig)
#             plt.xscale=10;plt.yscale=10;plt.xlim([0,8]);plt.ylim([0,8]);plt.xlabel('Ra Å');plt.ylabel('Rb Å')
#             R1,R2,heri_=para_[0]
#             Ra=R1*np.cos(np.radians(A2))-R2*np.sin((np.radians(A2)))
#             Rb=R2*np.cos(np.radians(A2))+R1*np.sin((np.radians(A2)))
            
#             df_A2=df_vdw[df_vdw['A2']==A2]
#             R_list=df_A2['R'].values
#             theta_list=df_A2['theta'].values
#             x_list=[R_list[i]*np.cos(np.radians(theta_list[i])) for i in range(len(theta_list))]
#             y_list=[R_list[i]*np.sin(np.radians(theta_list[i])) for i in range(len(theta_list))]
#             S_list=df_A2['S'].values
#             R1_min=np.round(x_list[np.argmin(S_list)]*np.cos(np.radians(A2))+y_list[np.argmin(S_list)]*np.sin((np.radians(A2))),1)
#             R2_min=np.round(y_list[np.argmin(S_list)]*np.cos(np.radians(A2))-x_list[np.argmin(S_list)]*np.sin((np.radians(A2))),1)
#             plt.title('heri={} S_min:{} R1_min={} R2_min={}'.format(A2*2,min(S_list),R1_min,R2_min))
#             plt.scatter(x_list,y_list,c=S_list)#,label='{}deg'.format(int(A2)))#,color=cm.jet(ind/len(self.A2_list)))#
#             plt.scatter(x_list[np.argmin(S_list)],y_list[np.argmin(S_list)],marker='D',color='blue',label='vdwmin')
#             plt.scatter(Ra,Rb,marker='D',color='red',label='opt')
#             plt.legend(bbox_to_anchor=(1,1),loc='upper left')
#         plt.show()
# #         #print('min:V={} Ra={} Rb={} R3={}'.format(min(V_list),Ra_list[np.argmin(V_list)],Rb_list[np.argmin(V_list)],R3_list[np.argmin(V_list)]))

# ###########################################
#     def make_csv_vdwmin(self):
#         df_vdw=pd.read_csv(self.path+self.name_csv)
#         df_contact=pd.DataFrame(columns=df_vdw.columns)
#         for A1 in self.A1_list:
#             for A2 in self.A2_list:
#                 if A1==0 and A2 >-90 :
#                     continue
#                 for A3 in self.A3_list:
#                     df_A=df_vdw[(df_vdw['A1']==A1)&(df_vdw['A2']==A2)&(df_vdw['A3']==A3)]
#                     S_list=df_A['S'].values
#                     idx_Smin=np.argmin(S_list)
#                     theta=df_A['theta'].values[idx_Smin]
#                     R=df_A['R'].values[idx_Smin]
#                     a=df_A['a'].values[idx_Smin]
#                     b=df_A['b'].values[idx_Smin]
#                     data=pd.Series([A1,A2,A3,theta,R,min(S_list),a,b],index=df_contact.columns)
#                     df_contact=df_contact.append(data,ignore_index=True)
#         df_contact.to_csv(self.path+'contact_6inter.csv',index=False)
        
#     def cmap_vdwmin(self,r):
#         plt.rcParams['font.size']=15
#         plt.rcParams['figure.figsize']=8,15
#         df_contact=pd.read_csv(self.path+'contact_6inter.csv')
#         A1_sc=df_contact['A1'].values
#         A2_sc=df_contact['A2'].values
#         A3_sc=df_contact['A3'].values
#         S_sc=df_contact['S'].values
#         x_sc,y_sc=[],[]
#         for ind,A1 in enumerate(A1_sc):
#             A1_rad=np.radians(A1)
#             A2_rad=np.radians(A2_sc[ind])
#             A3_rad=np.radians(A3_sc[ind])
#             x=np.sin(A1_rad)*np.cos(A2_rad)+r*np.cos(A2_rad+A3_rad)
#             y=np.sin(A1_rad)*np.sin(A2_rad)+r*np.sin(A2_rad+A3_rad)
#             x_sc.append(x)
#             y_sc.append(y)
#         plt.scatter(x_sc,y_sc,c=S_sc)
        
#     def cmap_vdwmin_A(self,argmin='A3'):
#         plt.rcParams['font.size']=15
#         plt.rcParams['figure.figsize']=8,15
#         df_contact=pd.read_csv(self.path+'contact_6inter.csv')
#         df_contact_A=pd.DataFrame(columns=df_contact.columns)
#         if argmin=='A3':
#             for A1 in self.A1_list:
#                 for A2 in self.A2_list:
#                     if A1==0 and A2 >-90 :
#                         continue
#                     df_A=df_contact[(df_contact['A1']==A1) & (df_contact['A2']==A2)]
#                     df_A=df_A.reset_index(drop=True)
#                     idx_Smin=np.argmin(df_A['S'])
#                     data=pd.Series(df_A.iloc[idx_Smin,:].values,index=df_contact_A.columns)
#                     df_contact_A=df_contact_A.append(data,ignore_index=True)
#         df_contact_A.to_csv(self.path+'contact_6inter_{}argmin.csv'.format(argmin),index=False)
        
    

In [8]:
vdw=vdw_vol()
vdw.make_csv_all('for_step2B+3.csv')

1.2820512820512822% completed
2.5641025641025643% completed
3.8461538461538463% completed
5.128205128205129% completed
6.410256410256411% completed
7.6923076923076925% completed
8.974358974358974% completed
10.256410256410257% completed
11.538461538461538% completed
12.820512820512821% completed
14.102564102564102% completed
15.384615384615385% completed
16.666666666666668% completed
17.94871794871795% completed
19.23076923076923% completed
20.512820512820515% completed
21.794871794871796% completed
23.076923076923077% completed
24.358974358974358% completed
25.641025641025642% completed
26.923076923076923% completed
28.205128205128204% completed
29.487179487179485% completed
30.76923076923077% completed
32.05128205128205% completed
33.333333333333336% completed
34.61538461538461% completed
35.8974358974359% completed
37.17948717948718% completed
38.46153846153846% completed
39.743589743589745% completed
41.02564102564103% completed
42.30769230769231% completed
43.58974358974359% compl